In [1]:
using Analytical
using Distributions
using StatsBase
using CSV
using Parsers

Loading empirical data from *mk_with_positions_BGS.txt*. Retrieving the total amount P and D of all proteins.

In [2]:
observedData = Analytical.readData("/home/jmurga/positiveSelectionHuman/201911/results/observedData.tsv")
observedData = observedData[size(observedData)[1],:]

2-element Array{Float64,1}:
 323808.0
  69660.0

Defining random values as paramateres and setting up estimating the binomial convolution to each B value

In [3]:
BRand = rand(append!(collect(0.25:0.05:0.95),0.999))
gam_negRand = -rand(80:200)
glRand = rand(10:20)
gHRand = rand(100:500)
alLowRand = rand(collect(0:0.1:0.4))
alTotRand = rand(collect(0.1:0.1:0.4))

0.3

In [4]:
Analytical.changeParameters(gam_neg=gam_negRand,gL=glRand,gH=gHRand,alLow=alLowRand,alTot=alTotRand,theta_f=1e-3,theta_mid_neutral=1e-3,al=0.184,be=0.000402,B=BRand,bRange=append!(collect(0.2:0.05:0.95),0.999),pposL=0.001,pposH=0,N=1000,n=661,Lf=10^6,rho=0.001,TE=5.0,convoluteBinomial=true)
adap

Analytical.parameters
  gam_neg: Int64 -158
  gL: Int64 16
  gH: Int64 397
  alLow: Float64 0.4
  alTot: Float64 0.3
  theta_f: Float64 0.001
  theta_mid_neutral: Float64 0.001
  al: Float64 0.184
  be: Float64 0.000402
  B: Float64 0.7
  bRange: Array{Float64}((17,)) [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.999]
  pposL: Float64 0.001
  pposH: Float64 0.0
  N: Int64 1000
  n: Int64 661
  Lf: Int64 1000000
  rho: Float64 0.001
  TE: Float64 5.0
  NN: Int64 2000
  nn: Int64 1322
  bn: Dict{Float64,Array{Float64,2}}


### **Sampling values from Poisson Distribution**

**Estimating manually $\alpha_{(x)}$ to check the Poisson sampling. The next analysis will be performed with the total sum of $D$ and $P$, taking into account all the proteins. However,** ```observedValue``` **at the next functions could be an array of observed values. The *dot* before any operation in Julia means *element-wise*, in this way we can sample in one $\alpha_{(x)}$ estimation any subset of protein data.**


In [5]:
# Setting up the analysis
Analytical.set_theta_f()
theta_f = adap.theta_f
adap.B = 0.999
Analytical.set_theta_f()
Analytical.setPpos()
adap.theta_f = theta_f
adap.B = BRand

0.7

#### **Expected fixation and poisson sampling**

```julia
function poissonFixation(;observedValue, λds, λdn)

    poissonS  = (λds/(λds + λdn) .* observedValue) .|> Poisson
    poissonD  = (λdn/(λds + λdn) .* observedValue) .|> Poisson

    sampledDs = rand.(poissonS,1)
    sampledDn = rand.(poissonD,1)

    return(reduce(vcat,sampledDs),reduce(vcat,sampledDn))
end
```

In [6]:
# Fixation
D = observedData[2]

fN    = adap.B*Analytical.fixNeut()*(adap.theta_mid_neutral/2.0)*adap.TE*adap.NN
fNeg  = adap.B*Analytical.fixNegB(0.5*adap.pposH+0.5*adap.pposL)*(adap.theta_mid_neutral/2.0)*adap.TE*adap.NN
fPosL = Analytical.fixPosSim(adap.gL,0.5*adap.pposL)*(adap.theta_mid_neutral/2.0)*adap.TE*adap.NN
fPosH = Analytical.fixPosSim(adap.gH,0.5*adap.pposH)*(adap.theta_mid_neutral/2.0)*adap.TE*adap.NN

ds = fN
dn = fNeg + fPosL + fPosH

0.0006446929366785066

In [7]:
expectedDs,expectedDn = Analytical.poissonFixation(observedValue=D, λds=ds, λdn=dn)
hcat(expectedDs,expectedDn)

1×2 Array{Int64,2}:
 34479  34878

#### **Expected polymorphism and poisson sampling**

**Option 1: getting the expected $Ps$ and $Pn$ for each expected value in $ps$, and $pn$. It means to get randomly $Ps$ and $Pn$ in each frequency using the total number of $P$. Initially, I did the same but I sampled the polymorphism using $ps$ and $pn$ as ```sum(ps)``` and  ```sum(pn)``` respectively.**

```julia
function poissonPolymorphism(;observedValues, λps, λpn)

    psPois(x,y=λps,z=λpn) = reduce(vcat,rand.((y./(y .+ z) .* x) .|> Poisson,1))
    pnPois(x,y=λps,z=λpn) = reduce(vcat,rand.((z./(y .+ z) .* x) .|> Poisson,1))
    
    sampledPs = observedValues .|> psPois # We can apply here any statistic measure
    sampledPn = observedValues .|> pnPois # We can apply here any statistic measure
    
    return sampledPs,sampledPn
end
```

In [8]:
# Polymorphism. Getting adap.*2 values. One per frequency estimated
P = observedData[1]

neut = Analytical.cumulativeSfs(Analytical.DiscSFSNeutDown())
selN = Analytical.cumulativeSfs(Analytical.DiscSFSSelNegDown(adap.pposH+adap.pposL))
sel = selN

ps = neut ./ (sel.+neut); ps = ps[1:end-1]  
pn = sel ./ (sel.+neut); pn = pn[1:end-1]

1321-element Array{Float64,1}:
 0.5790170953435917 
 0.5656628891077868 
 0.5541880016517515 
 0.5456884743660402 
 0.5393000713939385 
 0.5342358383671977 
 0.5300489821647711 
 0.5264855173444986 
 0.5233876787000579 
 0.5206504823616441 
 0.5182006042356017 
 0.5159848694705216 
 0.5139635175555217 
 ⋮                  
 0.41752252470151235
 0.41750885759459405
 0.4174952057785283 
 0.417481570734846  
 0.4174679547628089 
 0.41745436161278965
 0.41744079768074066
 0.417427274426912  
 0.41741381446546094
 0.4174004741063232 
 0.41738743467557765
 0.4173752560357667 

In [9]:
expectedPs, expectedPn = Analytical.poissonPolymorphism(observedValues = P, λps=ps, λpn=pn)
hcat(expectedPs,expectedPn)

1321×2 Array{Int64,2}:
 136040  187315
 140547  182794
 144310  179808
 146996  175958
 148814  173927
 151518  172495
 152605  171454
 153064  170424
 154318  169203
 154999  168746
 155454  167940
 157168  167091
 157000  165759
      ⋮        
 188301  135196
 189160  134868
 188345  135229
 188903  135088
 189148  135509
 188879  134887
 188359  134821
 187560  134989
 188071  134808
 188923  135012
 188783  135705
 188584  135239

In [10]:
mean(expectedPs)

182540.7047691143

In [11]:
mean(expectedPn)

141240.35579106738

**Option 2: sample the total number of $Pn$ and $Ps$ through the expected number of polymorphic sites by frequency category ($Pn_{(x)}$, $Ps_{(x)}$)**

```julia
function poissonPolymorphism2(;observedValues, λps, λpn)

    psPois(x,y=λps,z=λpn) = reduce(vcat,rand.((y./(y .+ z) .* x) .|> Poisson,1))
    pnPois(x,y=λps,z=λpn) = reduce(vcat,rand.((z./(y .+ z) .* x) .|> Poisson,1))
    
    sampledPs = observedValues .|> psPois # We can apply here any statistic measure
    sampledPn = observedValues .|> pnPois # We can apply here any statistic measure
    
    return sum(reduce(vcat,sampledPs)), sum(reduce(vcat,sampledPn))
end
```

In [ ]:
proteinData = CSV.read("/home/jmurga/mktest/data/mk_with_positions_BGS.txt",header=false,delim=' ')

In [13]:
function parseSfs(data,column)
    
    tmp = split.(data[:,column], ",")
    f(x) = Parsers.parse.(Float64,x[2:end-1])
    tmp = round.(reduce(vcat,tmp .|> f),digits=4) |> StatsBase.countmap
    
    x = zeros(adap.nn)
    for i in 1:adap.nn
        try
            x[i] = tmp[round.((i/adap.nn),digits=4)]
        catch
            x[i] = 0
        end
    end
    return(x)
end

parseSfs (generic function with 1 method)

Here I obtained the total polymorphism by frequency bins

In [14]:
Pn = parseSfs(proteinData,3)
Ps = parseSfs(proteinData,5)
Psfs = Pn .+ Ps; Psfs = Psfs[1:end-1]

1321-element Array{Float64,1}:
 151457.0
  34174.0
  16251.0
   9980.0
   7289.0
   5804.0
   4529.0
   3883.0
   3218.0
   3025.0
   2622.0
   2316.0
   2125.0
      ⋮  
     50.0
     45.0
     55.0
     70.0
     64.0
     91.0
     89.0
     94.0
    139.0
    171.0
    300.0
    883.0

In [15]:
expectedPs2, expectedPn2 = Analytical.poissonPolymorphism2(;observedValues=[Psfs], λps=ps, λpn=pn)

(150417, 178388)

Summary statistics: $Ds$, $Dn$, $Ps$ $Pn$, $\alpha_{(x\_withoutPositiveAlleles)}$

In [16]:
ret = 1 .- (fN/(fPosL + fPosH+  fNeg+0.0)) .* (sel./neut)
ret = ret[1:end-1]

summaryStatistics = hcat(expectedDs,expectedDn,expectedPs2,expectedPn2,ret[lastindex(ret)])

1×5 Array{Float64,2}:
 34479.0  34878.0  150417.0  178388.0  0.291622

### Checking in-built loop multithreading

In Julia you can easily parallelize a loop using ```$ export JULIA_NUM_THREADS=4```. Each iteration will be executed in a thread. In order to check the threads configured, just use in the julia console ```julia> Threads.nthreads()``` before the execution.

In [ ]:
a = zeros(20)
b = zeros(20)
@time Threads.@threads for i in 1:20
    a[i] = Threads.threadid()
    Analytical.changeParameters(gam_neg=-83,gL=10,gH=rand(100:500),alLow=0.2,alTot=0.2,theta_f=1e-3,theta_mid_neutral=1e-3,al=0.184,be=0.000402,B=0.999,bRange=[0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,0.999],pposL=0.001,pposH=0,N=1000,n=661,Lf=10^6,rho=0.001,TE=5.0,convoluteBinomial=true)
    b[i] = adap.gH
end

In [1]:
using Gadfly

SyntaxError: invalid syntax (<ipython-input-1-762049fda90d>, line 1)